In [29]:
# for debugging
import sys, importlib
if 'api' in sys.modules:
    importlib.reload(sys.modules['api'])
if 'csv_dumper' in sys.modules:
    importlib.reload(sys.modules['csv_dumper'])

# real
from api import RedditRequestManager
from csv_dumper import extract_comment_data, extract_post_data, extract_raw_data

In [30]:
reddit = RedditRequestManager(
    window_time_sec=5 * 60,
    max_requests_in_window=500,  # true limit is 1000, but we want to be carefuls
)

In [3]:
posts_res = reddit.search_posts("Coldplay", "stanford")

[RequestManager] sent query: https://oauth.reddit.com/r/Coldplay/search?q=stanford&sort_by=new&limit=100&restrict_sr=true
[RequestManager] sent query: https://oauth.reddit.com/r/Coldplay/search?q=stanford&sort_by=new&limit=100&restrict_sr=true&after=t3_1l0w3kh
[RequestManager] catching up on metadata, requests_used=2, requests_remaining=998, time_until_window_reset=229
[RequestManager] Submissions query for subreddit_name='Coldplay', query_term='stanford' yielded 123 submissions


In [31]:
comments_res = reddit.get_comments("1lhoxch")

[RequestManager] sent query: https://oauth.reddit.com/comments/1lhoxch?limit=500&sort=old&depth=10
[RequestManager] catching up on metadata, requests_used=5, requests_remaining=995, time_until_window_reset=329
[RequestManager] sent query: https://oauth.reddit.com/api/morechildren?link_id=t3_1lhoxch&limit_children=false&sort=old&api_type=json&children=mz6qy4g,mz6d807,mz5zqk1,mz6cdqz,mz5wfnn,mz69drd,mz69lp2,mz64xcx,mz6ejnf,mz6fw11,mz6e7iq,mz6kckn,mz728ff,mz7jjes,mz5xoyd,mz5wkj9,mz6dq2d,mz5znhy,mz5zz6f,mz63n73,mz673og,mz6oo8i,mz7b9do,mzald9x,mz7193w,mz5ze2p,mz6x6w3,mz6f0dx,mz9dmp2,mz7u315,mz6jtws,mz695is,mz9dt1n,mz7kmph,mz7gnfe,mzcykn4,mz69zkl,mz6buml,mz6a4fi,mz61sqq,mz6corx,mz64rw4,mz6oz3k,mz6ccl9,mz6hqxt,mz6qmca,mz84wxq,mz5x90s,mz6k5qc,mz62ero,mz9iha3,mz7w9j4,mz6bf4z,mz94oip,mz6ux30,mz7dade,mz6r8ph,mz6i9e2,mza6ttr,mz7jrbt,mz5xpr5,mz61fm6,mz66hrr,mz6er71,mz6w37c,mz6ryas,mz7fwbr,mz813hd,mzf8arl,mz6smm5,mz6zrw4,mz7i7nu,mz89gxp,mz8dax8,mz8xeum,mzcg6qo,mz66u1s,mz688v8,mz62bcf,mz7jss7,mzmj9c3

In [ ]:
import json

# it's always good to dump the data that we've already saved,
# so don't have to make the same queries more than onces
extract_raw_data(posts_res, "coldplay-stanford-raw-posts.json")

# and reload them from the files when necessary on-demand
with open("coldplay-stanford-raw-posts.json") as f:
    loaded_posts_res = json.load(f)

# if they're equal, there should be no error here
assert loaded_posts_res == posts_res

True

In [90]:
extract_post_data(posts_res, "coldplay-stanford.csv")

In [32]:
extract_comment_data(comments_res, "gaming-post-comments.csv")